In [2]:
import tensorflow as tf
import numpy as np
import functools
from tqdm import tqdm 
import pandas as pd
import sklearn

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras import metrics, losses
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('./data/bubble_detection.csv')

In [129]:
scaler = MinMaxScaler(feature_range=(0, 1))
features = scaler.fit_transform(df.iloc[:,1:8])
output = np.array(df.iloc[:,9], dtype='float64')

In [152]:
time_steps = 36
x = []
y = []
for row in range(len(features) - time_steps):
    x.append(features[row : row + time_steps])
    y.append(output[row + time_steps - 1])

In [153]:
x = np.array(x, dtype = 'float64')
y = np.array(y, dtype = 'float64')

In [154]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=202144)

In [155]:
1 - sum(y_train) / len(y_train)

0.9460431654676259

In [156]:
model = Sequential([
    Bidirectional(LSTM(units=50,
                    input_shape=(x_train.shape[1], x_train.shape[2]),
                    recurrent_activation='sigmoid',
                    recurrent_initializer='glorot_uniform')),
    Dense(units=1, activation='sigmoid')
])

In [157]:
model.compile(optimizer=Adam(learning_rate=1e-2),
            loss=binary_focal_loss(alpha=0.75),
            metrics=[metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall()])

In [158]:
model.fit(x_train, y_train)

Train on 556 samples
556/556 [==============================] - 4s 8ms/sample - loss: 0.0343 - binary_accuracy: 0.8921 - precision_13: 0.0312 - recall_13: 0.0333


In [159]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_7 (Bidirection multiple                  23200     
_________________________________________________________________
dense_8 (Dense)              multiple                  101       
Total params: 23,301
Trainable params: 23,301
Non-trainable params: 0
_________________________________________________________________


In [144]:
yhat = model.predict_classes(x_test)

In [160]:
model.predict(x_test)

array([[0.33441395],
       [0.30302417],
       [0.3417294 ],
       [0.3151436 ],
       [0.32966316],
       [0.33544472],
       [0.31353974],
       [0.33672625],
       [0.29482037],
       [0.32858953],
       [0.32807076],
       [0.31951374],
       [0.32996422],
       [0.31622523],
       [0.33283246],
       [0.3225739 ],
       [0.33260208],
       [0.32405582],
       [0.33006313],
       [0.31750435],
       [0.31862375],
       [0.3272368 ],
       [0.31491625],
       [0.33387563],
       [0.29625887],
       [0.3176946 ],
       [0.33267063],
       [0.30956635],
       [0.29439116],
       [0.34643626],
       [0.2993663 ],
       [0.3310492 ],
       [0.32779664],
       [0.33448997],
       [0.3090125 ],
       [0.33685058],
       [0.33924505],
       [0.3554514 ],
       [0.33128998],
       [0.30531853],
       [0.33486265],
       [0.3353409 ],
       [0.31905496],
       [0.3280389 ],
       [0.33572602],
       [0.3160811 ],
       [0.3229942 ],
       [0.331

In [93]:
sklearn.metrics.confusion_matrix(y_test, yhat)

array([[134,   0],
       [  6,   0]])

In [42]:
# imported from: https://github.com/aldi-dimara/keras-focal-loss/blob/master/focal_loss.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 19 08:20:58 2018
@OS: Ubuntu 18.04
@IDE: Spyder3
@author: Aldi Faizal Dimara (Steam ID: phenomos)
"""

import tensorflow.keras.backend as K
import tensorflow as tf

def binary_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha_t*((1-p_t)^gamma)*log(p_t)
        
        p_t = y_pred, if y_true = 1
        p_t = 1-y_pred, otherwise
        
        alpha_t = alpha, if y_true=1
        alpha_t = 1-alpha, otherwise
        
        cross_entropy = -log(p_t)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    def focal_loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1-y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true)*alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1-alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1-p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
    
    return focal_loss